In [39]:
# TODO: refactor this into utility

import random
from itertools import chain
import pandas as pd
import networkx as nx
from itertools import combinations  
import requests 
import numpy as np
# from datetime import datetime, date, timedelta

class DataGenerator:
    def __init__(self,num_docs=10,num_patients=100,min_notes_per_discharge=5,max_notes_per_discharge=10,
                 max_stay_per_patient=6,start=None,end=None):
        self.MIN_NOTES_PER_DIS = min_notes_per_discharge
        self.MAX_NOTES_PER_DIS = max_notes_per_discharge
        self.MAX_STAY_PER_PATIENT = max_stay_per_patient     
        self.drs = [name for name in self._get_random_names(num_docs)]
        self.drs_non_collab = [name for name in self._get_random_names(num_docs)]
        self.patients = [name for name in self._get_random_names(num_patients, "patient")]    
        self.end = np.datetime64('now') if end == None else np.datetime64(end)
        self.start = self.end - np.timedelta64(365,'D') if start == None else np.datetime64(start)
        self._create_date_range()
        self._generate_data()

    def _get_random_names(self,count, prefix="dr"):
#         response = requests.get(f'https://randomuser.me/api?results={count}&inc=name&nat=us,gb,es')
#         names = [f"{user['name']['first']} {user['name']['last']}" for user in response.json()['results']]
        names = [f"{prefix}{i+1}" for i in range(0,count)]
        return names

    def _get_date(self):
        return random.choice(self.date_range) + np.timedelta64(random.randint(1,86400),'s')

    def _generate_notes_for_discharge(self, discharge):
#         disposition = discharge[4]
#         dr_list = self.drs if disposition < 1 else self.drs_non_collab
        dr_list = random.choices(self.drs,k=random.randint(6,10))
        notes = [(discharge[0], random.choice(dr_list), discharge[2] + self._get_random_timedelta()) 
                 for _ in range(0, random.randint(self.MIN_NOTES_PER_DIS,self.MAX_NOTES_PER_DIS))]
        return notes

    def _create_date_range(self):
        dr = np.arange(self.start, self.end, np.timedelta64(1, 'D'))
        self.date_range = dr
#         self.date_range = [date for date in dr.astype(object)]

    def _generate_notes(self,discharges):
        for d in discharges:
            yield self._generate_notes_for_discharge(d)

    def _get_random_timedelta(self):
        return np.timedelta64(random.randint(1,self.MAX_STAY_PER_PATIENT - 1), 'D') + np.timedelta64(random.randint(1,86400),'s')
    
    def _flip(self):
        return 0 if random.random() > .1 else 1

    def _generate_data(self):  
        discharges = [(i, p, d := self._get_date(), d + np.timedelta64(self.MAX_STAY_PER_PATIENT - 1,'D') + np.timedelta64(random.randint(1,86400),'s'), self._flip(), random.randint(65,80))
            for i,p in enumerate(self.patients)]

        notes = list(chain.from_iterable(self._generate_notes(discharges)))
        
        dis_columns = ['id', 'patient', 'arrive_date', 'discharge_date', 'disposition', 'age']

        self.dis_df = pd.DataFrame(discharges, columns=dis_columns)

        note_columns=['discharge_id', 'dr', 'date']
        
        self.note_df = pd.DataFrame(notes, columns=note_columns)
    
if __name__ == "__main__":
    data = DataGenerator(num_docs=30, num_patients=70, max_stay_per_patient=7,min_notes_per_discharge=15,max_notes_per_discharge=18)
    print(data.note_df.head())
    print(data.note_df.shape)
    print(data.dis_df.head())
    print(data.dis_df.shape)
    
    data.note_df.to_csv('../data/notes_w_datetime.csv', index=False)
    data.dis_df.to_csv('../data/discharges_w_datetime.csv', index=False)



   discharge_id    dr                date
0             0   dr4 2020-11-13 19:15:09
1             0  dr19 2020-11-13 10:19:28
2             0  dr19 2020-11-15 17:09:15
3             0   dr4 2020-11-10 16:48:03
4             0  dr26 2020-11-13 13:41:09
(1143, 3)
   id   patient         arrive_date      discharge_date  disposition  age
0   0  patient1 2020-11-09 12:46:46 2020-11-15 16:38:33            0   72
1   1  patient2 2020-06-27 04:17:28 2020-07-03 12:24:36            0   73
2   2  patient3 2020-10-27 11:28:04 2020-11-02 19:08:36            0   67
3   3  patient4 2020-07-19 16:11:45 2020-07-25 22:53:00            0   71
4   4  patient5 2020-12-05 00:33:37 2020-12-11 14:11:50            0   76
(70, 6)


In [ ]:
def get_random_names(count, prefix="dr"):
    names = [f"{prefix}{i}" for i in range(1,count)]
    return names

names = get_random_names(10)

choices = random.choices(names,k=5)

print(choices)

In [38]:
import numpy as np
import datetime

# end = np.datetime64('now')
# start = end - np.timedelta64(365,'D')
# dr = np.arange(start, end, np.timedelta64(1, 'D'))
# rd = np.random.choice(dr) + np.timedelta64(np.random.choice(range(1, 1440)),'s')

# print(rd)
now = np.datetime64('now')
later = now + np.timedelta64(np.random.choice(range(1, 86400)),'s')
print(later)

2021-05-05T12:16:49
